In [4]:
import cv2
import matplotlib.pyplot as plt

In [19]:
image = cv2.imread("0Q6A9930.JPG")
image = cv2.resize(image,(720,640))

face_pbtxt="models/opencv_face_detector.pbtxt"
face_pb="models/opencv_face_detector_uint8.pb"
age_model="models/age_net.caffemodel"
age_prototxt="models/age_deploy.prototxt"
gender_prototxt="models/gender_deploy.prototxt"
gender_model="models/gender_net.caffemodel"


In [20]:
#Load Models
face=cv2.dnn.readNet(face_pb,face_pbtxt)
age=cv2.dnn.readNet(age_model,age_prototxt)
gender=cv2.dnn.readNet(gender_model,gender_prototxt)

In [7]:
#Setup Classifications
age_classification=['(0-2)','(3-6)','(8-12)','(15-20)','(25-32)','(38-43)','(48-53)','(60-100)']
gender_classification=['Male','Female']

In [21]:
#Image Copy
img_cp=image.copy()

img_height=img_cp.shape[0]
img_width=img_cp.shape[1]
blob = cv2.dnn.blobFromImage(img_cp,1.0,(300,300),[104,117,123],True,False)


face.setInput(blob)
detected_faces=face.forward()
# print(f"Confidence: {detected_faces[0,0]}")

print("Blob shape:", blob.shape)
print("Detected faces shape:", detected_faces.shape)

Blob shape: (1, 3, 300, 300)
Detected faces shape: (1, 1, 200, 7)


In [22]:
face_bounds=[]
for i in range(detected_faces.shape[2]):
    confidence=detected_faces[0,0,i,2]
    if (confidence>0.9):
        print("Detected Faces: ",detected_faces[0,0,i],)
        x1=int(detected_faces[0,0,i,3]*img_width)
        y1=int(detected_faces[0,0,i,4]*img_height)
        x2=int(detected_faces[0,0,i,5]*img_width)
        y2=int(detected_faces[0,0,i,6]*img_height)
        # print([x1,y1,x2,y2])
        cv2.rectangle(img_cp,(x1,y1),(x2,y2),(0,0,255),2)
        print(f"Confidence: {confidence:.3f} | Face at: ({x1}, {y1}) to ({x2}, {y2})")
        face_bounds.append([x1,y1,x2,y2])

Detected Faces:  [0.         1.         0.9842712  0.48270634 0.26641303 0.6934413
 0.43035346]
Confidence: 0.984 | Face at: (347, 170) to (499, 275)


In [24]:
print("face_bounds: ",face_bounds)

for face_bound in face_bounds:
    try:
        face=img_cp[max(0,face_bound[1]-15): min(face_bound[3]+15,img_cp.shape[0]-1),
                    max(0,face_bound[0]-15): min(face_bound[2]+15,img_cp.shape[1]-1)]
        
        print("Face",face[0])
    
        blob= cv2.dnn.blobFromImage(face, 1.0, (227,227),[104,117,123],True)
        gender.setInput(blob)
        predicted_gender=gender.forward()
        _gender= gender_classification[predicted_gender[0].argmax()]
        print("Gender :",_gender)

        age.setInput(blob)
        age_prediction=age.forward()
        _age=age_classification[age_prediction[0].argmax()]
        print("AGE :",_age)
    except Exception as e:
        print(e)

face_bounds:  [[347, 170, 499, 275]]
Face [[ 75  76  80]
 [ 74  75  79]
 [ 82  83  87]
 [ 61  59  65]
 [ 87  87  93]
 [ 84  80  82]
 [ 82  76  81]
 [ 88  84  89]
 [ 89  86  88]
 [117 112 113]
 [ 92  86  87]
 [ 99  92  95]
 [ 99  96  98]
 [ 80  75  77]
 [113 106 109]
 [110 104 108]
 [ 74  70  69]
 [ 67  62  64]
 [ 97  95  95]
 [ 73  68  68]
 [ 88  86  86]
 [ 85  80  80]
 [ 69  64  63]
 [ 94  92  91]
 [ 95  91  93]
 [ 99  94  95]
 [ 96  94  93]
 [ 98  98  98]
 [ 54  49  50]
 [ 46  41  42]
 [ 70  65  67]
 [ 45  41  42]
 [ 78  74  74]
 [111 106 108]
 [ 87  85  87]
 [ 91  86  87]
 [ 89  83  85]
 [ 85  78  80]
 [103  96  99]
 [ 93  90  92]
 [ 83  78  79]
 [ 40  35  39]
 [ 85  83  83]
 [ 74  74  74]
 [ 20  20  20]
 [ 76  74  74]
 [ 90  87  89]
 [ 63  60  62]
 [ 95  95  96]
 [ 96  94  98]
 [ 90  89  91]
 [ 59  60  61]
 [ 58  57  59]
 [ 58  59  63]
 [ 41  39  43]
 [ 63  62  66]
 [ 48  47  53]
 [ 64  65  69]
 [ 97  97 103]
 [ 77  77  82]
 [ 76  78  83]
 [ 99 102 107]
 [ 84  87  92]
 [ 84  84  91

In [ ]:
while True:
    cv2.imshow("result",img_cp)
    k = cv2.waitKey(1)
    if k == ord('q'):
        break

cv2.destroyAllWindows()